# **Tworzenie API we Flasku – Wprowadzenie**

W tym ćwiczeniu nauczysz się, jak stworzyć proste API w Flasku, uruchomić je, wysyłać do niego zapytania oraz wykorzystać model decyzyjny w oparciu o podstawową regułę logiczną.

## **1️⃣ Tworzenie podstawowego API**
Najpierw utworzymy podstawową aplikację Flask.

### **Zapisanie kodu API do pliku**
W Jupyter Notebooku użyj magicznej komendy `%%file`, aby zapisać kod podstawowej aplikacji flask do pliku `app.py`: Kod znajdziesz na [cw1](https://sebkaz-teaching.github.io/RTA_2025/cw1.html)
Jako tekst do wyświetlenie strony głównej użyj `Witaj w moim API!`.

In [1]:
%%file app.py
from flask import Flask, jsonify

app = Flask(__name__)

@app.route('/')
def home():
    return jsonify({"message": "Witaj w moim API!"})

if __name__ == '__main__':
    app.run()

Overwriting app.py


Teraz uruchom API w terminalu, wpisując:
```sh
python app.py
```
Flask uruchomi serwer lokalnie pod adresem `http://127.0.0.1:5000/`.

### **Sprawdzenie działania API**
W Jupyter Notebooku wykonaj zapytanie GET do strony głównej. Na podstawie pola `status_code` napisz wyrażenie warunkowe które dla status_code 200 wyświetli zawartość odpowiedzi (z pola `content`).

In [3]:
import requests
response = requests.get("http://127.0.0.1:5000/")

if response.status_code == 200:
    print(response.json()["message"])

Witaj w moim API!


Jeśli wszystko działa poprawnie, zobaczysz komunikat `Witaj w moim API!`.
---

## **2️⃣ Dodanie nowej podstrony**
Dodajmy nową podstronę `mojastrona`, która zwróci komunikat `To jest moja strona!`.

In [4]:
%%file app.py
from flask import Flask, jsonify

app = Flask(__name__)

@app.route('/')
def home():
    return jsonify({"message": "Witaj w moim API!"})

@app.route('/mojastrona')
def mojastrona():
    return jsonify({"message": "To jest moja podstrona!"})

if __name__ == '__main__':
    app.run()

Overwriting app.py


Ponownie uruchom API i wykonaj zapytanie do strony `"http://127.0.0.1:5000/mojastrona"`:

In [ ]:
#info: musisz ponownie uruchomić api czyli python app.py

In [24]:
response = requests.get("http://127.0.0.1:5000/mojastrona")

if response.status_code == 200:
    print(response.json()["message"])

To jest moja podstrona!


Powinieneś zobaczyć: `To jest moja strona!`

---

## **3️⃣ Automatyczne uruchamianie serwera z Jupyter Notebook**
Zamknij wcześniej uruchomiony serwer (`Ctrl+C` w terminalu) i uruchom go ponownie bezpośrednio z Jupyter Notebook, korzystając z `subprocess.Popen`:

In [6]:
import subprocess
server = subprocess.Popen(["python", "app.py"])

In [7]:
!pip install pytest requests -q

In [8]:
%%file test_app.py
import pytest
import requests

def test_podstr():
    response = requests.get("http://127.0.0.1:5000/")
    assert response.status_code == 200
    assert response.json()["message"]

Overwriting test_app.py


In [9]:
!pytest test_app.py

============================= test session starts ==============================
platform linux -- Python 3.11.6, pytest-8.3.5, pluggy-1.5.0
rootdir: /home/jovyan/notebooks/ogr/laboratorium1_2
plugins: anyio-4.0.0
collected 1 item                                                               

test_app.py .                                                            [100%]

============================== 1 passed in 0.34s ===============================


Po testach zamknij serwer wykorzystując metodę `kill`:


In [10]:
server.kill()

---

## **4️⃣ Obsługa parametrów w adresie URL**
Dodajemy nową podstronę `/hello`, która będzie przyjmować parametr `name`.

Edytuj `app.py`, dodając odpowiedni  kod


In [27]:
%%file app.py
from flask import Flask, jsonify, request

app = Flask(__name__)

@app.route('/')
def home():
    return jsonify({"message": "Witaj w moim API!"})

@app.route('/mojastrona')
def mojastrona():
    return jsonify({"message": "To jest moja podstrona!"})

@app.route('/hello')
def hellopage():
    name = request.args.get('name')
    if name:
        return jsonify({"message": f"Hello {name}!"})
    else:
        return jsonify({"message": "Hello!"})
        
if __name__ == '__main__':
    app.run()

Overwriting app.py


Uruchom serwer i sprawdź działanie API:
```python
res1 = requests.get("http://127.0.0.1:5000/hello")
print(res1.content)  # Powinno zwrócić "Hello!"

res2 = requests.get("http://127.0.0.1:5000/hello?name=Sebastian")
print(res2.content)  # Powinno zwrócić "Hello Sebastian!"
```

---

In [31]:
server = subprocess.Popen(["python", "app.py"])

In [32]:
res1 = requests.get("http://127.0.0.1:5000/hello")
print(res1.content)  # Powinno zwrócić "Hello!"

b'{"message":"Hello!"}\n'


In [33]:
res2 = requests.get("http://127.0.0.1:5000/hello?name=Sebastian")
print(res2.content)  # Powinno zwrócić "Hello Sebastian!"

b'{"message":"Hello Sebastian!"}\n'


In [35]:
server.kill()

## **5️⃣ Tworzenie API z prostym modelem ML**
Stworzymy nową podstronę `/api/v1.0/predict`, która przyjmuje dwie liczby i zwraca wynik reguły decyzyjnej:
- Jeśli suma dwóch liczb jest większa niż 5.8, zwraca `1`.
- W przeciwnym razie zwraca `0`.


In [36]:
%%file app.py
from flask import Flask, jsonify, request

app = Flask(__name__)

@app.route('/')
def home():
    return jsonify({"message": "Witaj w moim API!"})

@app.route('/mojastrona')
def mojastrona():
    return jsonify({"message": "To jest moja podstrona!"})

@app.route('/hello')
def hellopage():
    name = request.args.get('name')
    if name:
        return jsonify({"message": f"Hello {name}!"})
    else:
        return jsonify({"message": "Hello!"})

@app.route('/api/v1.0/predict')
def predict():
    num1 = request.args.get('num1', type=float)
    num2 = request.args.get('num2', type=float)
    if (num1+num2) > 5.8:
        prediction = 1
    else:
        prediction = 0
        
    return jsonify({
        "prediction": prediction,
        "features": {
            "num1": num1,
            "num2": num2
        }
    })
        
if __name__ == '__main__':
    app.run()

Overwriting app.py


In [39]:
server = subprocess.Popen(["python", "app.py"])

In [40]:
res = requests.get("http://127.0.0.1:5000/api/v1.0/predict?num1=1&num2=4")
print(res.json())  # Powinno zwrócić {"prediction": 1, "features": {"num1": 3.0, "num2": 4.0}}

{'features': {'num1': 1.0, 'num2': 4.0}, 'prediction': 0}


In [41]:
server.kill()

Sprawdź działanie API:
```python
res = requests.get("http://127.0.0.1:5000/api/v1.0/predict?num1=3&num2=4")
print(res.json())  # Powinno zwrócić {"prediction": 1, "features": {"num1": 3.0, "num2": 4.0}}
```

---

## **Podsumowanie**
Po wykonaniu tego ćwiczenia studenci będą umieli:
✅ Tworzyć podstawowe API w Flasku.  
✅ Dodawać podstrony i obsługiwać parametry URL.  
✅ Wysyłać zapytania GET i analizować odpowiedzi.  
✅ Automatycznie uruchamiać serwer z Jupyter Notebook.  
✅ Implementować prosty model decyzyjny w API.  

Gotowi na kolejne wyzwania? 🚀

In [42]:
!pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.
